Overview

Business Understanding
    Include stakeholder and key business questions

Data Understanding and Analysis
    Source of data

Description of data
    Three visualizations (the same visualizations presented in the slides and notebook)

Conclusion
    Summary of conclusions including three relevant findings

Outline From https://github.com/learn-co-curriculum/dsc-project-template/tree/example-aab

Overview
For this project, you will use exploratory data analysis to generate insights for a business stakeholder.

Business Problem
Microsoft sees all the big companies creating original video content and they want to get in on the fun. 
They have decided to create a new movie studio, but they don’t know anything about creating movies. 
You are charged with exploring what types of films are currently doing the best at the box office. 
You must then translate those findings into actionable insights that the head of Microsoft's new movie studio 
can use to help decide what type of films to create.

Data
1. directors influence the performance of the movie
    profit of each directors
    not median bc of how long they have been directing
    mean can be 
2. genres to go into
    look at directors that do well in these genres
3. 

Methods
- standard for performance = high imdb score and ticket sales
- directros 


Results

Conclusions

#maybe
Next Step

Repository structure

Thank you

In [1]:
### IMPORTS
import pandas as pd
import matplotlib.pyplot as plt
import sqlite3
%matplotlib inline

### DATAFRAME DEFINITIONS
# define all dataframes for later
gross_df = pd.read_csv('data/zipped_files/bom.movie_gross.csv.gz')
movies_df = pd.read_csv('data/zipped_files/tmdb.movies.csv.gz')
budgets_df = pd.read_csv('data/zipped_files/tn.movie_budgets.csv.gz')
# tsv files have to be read with \t (tab spcaes) as their delimiter
info_df = pd.read_csv('data/zipped_files/rt.movie_info.tsv.gz',delimiter='\t')
# file has an issue with encoding, read as cp850 or it throws an error
reviews_df = pd.read_csv('data/zipped_files/rt.reviews.tsv.gz', delimiter='\t', encoding='cp850')

In [36]:
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 1000)

In [3]:
!ls data/unzipped

im.db


In [4]:
conn = sqlite3.connect('data/unzipped/im.db')

In [5]:
cursor = conn.cursor()

In [6]:
cursor.fetchall()
#will return empty list bc nothing is executed as a query

[]

In [9]:
#to see all of the tables in the sql schema

schema_df = pd.read_sql("""

SELECT *
FROM sqlite_master

""", conn)

schema_df

,type,name,tbl_name,rootpage,sql
0,table,movie_basics,movie_basics,2,"CREATE TABLE ""movie_basics"" (\n""movie_id"" TEXT..."
1,table,directors,directors,3,"CREATE TABLE ""directors"" (\n""movie_id"" TEXT,\n..."
2,table,known_for,known_for,4,"CREATE TABLE ""known_for"" (\n""person_id"" TEXT,\..."
3,table,movie_akas,movie_akas,5,"CREATE TABLE ""movie_akas"" (\n""movie_id"" TEXT,\..."
4,table,movie_ratings,movie_ratings,6,"CREATE TABLE ""movie_ratings"" (\n""movie_id"" TEX..."
5,table,persons,persons,7,"CREATE TABLE ""persons"" (\n""person_id"" TEXT,\n ..."
6,table,principals,principals,8,"CREATE TABLE ""principals"" (\n""movie_id"" TEXT,\..."
7,table,writers,writers,9,"CREATE TABLE ""writers"" (\n""movie_id"" TEXT,\n ..."


![database schema flow chart](./images/db_schema.jpeg "Flowchart for data from IMDB is located in a SQLite database")

In [10]:
print(schema_df['sql'].iloc[0])

CREATE TABLE "movie_basics" (
"movie_id" TEXT,
  "primary_title" TEXT,
  "original_title" TEXT,
  "start_year" INTEGER,
  "runtime_minutes" REAL,
  "genres" TEXT
)


In [27]:
movie_basics = pd.read_sql("""

SELECT *
FROM movie_basics

""", conn)

movie_basics

,movie_id,primary_title,original_title,start_year,runtime_minutes,genres
0,tt0063540,Sunghursh,Sunghursh,2013,175.0,"Action,Crime,Drama"
1,tt0066787,One Day Before the Rainy Season,Ashad Ka Ek Din,2019,114.0,"Biography,Drama"
2,tt0069049,The Other Side of the Wind,The Other Side of the Wind,2018,122.0,Drama
3,tt0069204,Sabse Bada Sukh,Sabse Bada Sukh,2018,NaN,"Comedy,Drama"
4,tt0100275,The Wandering Soap Opera,La Telenovela Errante,2017,80.0,"Comedy,Drama,Fantasy"
...,...,...,...,...,...,...
146139,tt9916538,Kuambil Lagi Hatiku,Kuambil Lagi Hatiku,2019,123.0,Drama
146140,tt9916622,Rodolpho Teóphilo - O Legado de um Pioneiro,Rodolpho Teóphilo - O Legado de um Pioneiro,2015,NaN,Documentary
146141,tt9916706,Dankyavar Danka,Dankyavar Danka,2013,NaN,Comedy
146142,tt9916730,6 Gunn,6 Gunn,2017,116.0,None


In [26]:
movie_akas = pd.read_sql("""

SELECT *
FROM movie_akas

""", conn)

movie_akas

,movie_id,ordering,title,region,language,types,attributes,is_original_title
0,tt0369610,10,Джурасик свят,BG,bg,None,None,0.0
1,tt0369610,11,Jurashikku warudo,JP,None,imdbDisplay,None,0.0
2,tt0369610,12,Jurassic World: O Mundo dos Dinossauros,BR,None,imdbDisplay,None,0.0
3,tt0369610,13,O Mundo dos Dinossauros,BR,None,None,short title,0.0
4,tt0369610,14,Jurassic World,FR,None,imdbDisplay,None,0.0
...,...,...,...,...,...,...,...,...
331698,tt9827784,2,Sayonara kuchibiru,None,None,original,None,1.0
331699,tt9827784,3,Farewell Song,XWW,en,imdbDisplay,None,0.0
331700,tt9880178,1,La atención,None,None,original,None,1.0
331701,tt9880178,2,La atención,ES,None,None,None,0.0


In [ ]:
imdb_directors = pd.read_sql("""

SELECT d.movie_id, p.primary_name FROM directoras as d
LEFT JOIN persons as p
    ON d person_id = 

""")

In [11]:
directors = pd.read_sql("""

SELECT *
FROM directors

""", conn)

directors

,movie_id,person_id
0,tt0285252,nm0899854
1,tt0462036,nm1940585
2,tt0835418,nm0151540
3,tt0835418,nm0151540
4,tt0878654,nm0089502
...,...,...
291169,tt8999974,nm10122357
291170,tt9001390,nm6711477
291171,tt9001494,nm10123242
291172,tt9001494,nm10123248


In [12]:
persons = pd.read_sql("""

SELECT *
FROM persons

""", conn)

persons

,person_id,primary_name,birth_year,death_year,primary_profession
0,nm0061671,Mary Ellen Bauder,NaN,NaN,"miscellaneous,production_manager,producer"
1,nm0061865,Joseph Bauer,NaN,NaN,"composer,music_department,sound_department"
2,nm0062070,Bruce Baum,NaN,NaN,"miscellaneous,actor,writer"
3,nm0062195,Axel Baumann,NaN,NaN,"camera_department,cinematographer,art_department"
4,nm0062798,Pete Baxter,NaN,NaN,"production_designer,art_department,set_decorator"
...,...,...,...,...,...
606643,nm9990381,Susan Grobes,NaN,NaN,actress
606644,nm9990690,Joo Yeon So,NaN,NaN,actress
606645,nm9991320,Madeline Smith,NaN,NaN,actress
606646,nm9991786,Michelle Modigliani,NaN,NaN,producer


In [16]:
movies_df.head(20)

,Unnamed: 0,genre_ids,id,original_language,original_title,popularity,release_date,title,vote_average,vote_count
0,0,"[12, 14, 10751]",12444,en,Harry Potter and the Deathly Hallows: Part 1,33.533,2010-11-19,Harry Potter and the Deathly Hallows: Part 1,7.7,10788
1,1,"[14, 12, 16, 10751]",10191,en,How to Train Your Dragon,28.734,2010-03-26,How to Train Your Dragon,7.7,7610
2,2,"[12, 28, 878]",10138,en,Iron Man 2,28.515,2010-05-07,Iron Man 2,6.8,12368
3,3,"[16, 35, 10751]",862,en,Toy Story,28.005,1995-11-22,Toy Story,7.9,10174
4,4,"[28, 878, 12]",27205,en,Inception,27.920,2010-07-16,Inception,8.3,22186
5,5,"[12, 14, 10751]",32657,en,Percy Jackson & the Olympians: The Lightning T...,26.691,2010-02-11,Percy Jackson & the Olympians: The Lightning T...,6.1,4229
6,6,"[28, 12, 14, 878]",19995,en,Avatar,26.526,2009-12-18,Avatar,7.4,18676
7,7,"[16, 10751, 35]",10193,en,Toy Story 3,24.445,2010-06-17,Toy Story 3,7.7,8340
8,8,"[16, 10751, 35]",20352,en,Despicable Me,23.673,2010-07-09,Despicable Me,7.2,10057
9,9,"[16, 28, 35, 10751, 878]",38055,en,Megamind,22.855,2010-11-04,Megamind,6.8,3635


In [37]:
gross_df

,title,studio,domestic_gross,foreign_gross,year
0,Toy Story 3,BV,415000000.0,652000000,2010
1,Alice in Wonderland (2010),BV,334200000.0,691300000,2010
2,Harry Potter and the Deathly Hallows Part 1,WB,296000000.0,664300000,2010
3,Inception,WB,292600000.0,535700000,2010
4,Shrek Forever After,P/DW,238700000.0,513900000,2010
...,...,...,...,...,...
3382,The Quake,Magn.,6200.0,NaN,2018
3383,Edward II (2018 re-release),FM,4800.0,NaN,2018
3384,El Pacto,Sony,2500.0,NaN,2018
3385,The Swan,Synergetic,2400.0,NaN,2018


In [34]:
list(gross_df['title'])

['Toy Story 3',
 'Alice in Wonderland (2010)',
 'Harry Potter and the Deathly Hallows Part 1',
 'Inception',
 'Shrek Forever After',
 'The Twilight Saga: Eclipse',
 'Iron Man 2',
 'Tangled',
 'Despicable Me',
 'How to Train Your Dragon',
 'Clash of the Titans (2010)',
 'The Chronicles of Narnia: The Voyage of the Dawn Treader',
 "The King's Speech",
 'Tron Legacy',
 'The Karate Kid',
 'Prince of Persia: The Sands of Time',
 'Black Swan',
 'Megamind',
 'Robin Hood',
 'The Last Airbender',
 'Little Fockers',
 'Resident Evil: Afterlife',
 'Shutter Island',
 'Salt',
 'Sex and the City 2',
 'The Tourist',
 'The Expendables',
 'Grown Ups',
 'Knight & Day',
 'True Grit',
 "Gulliver's Travels",
 'Percy Jackson & The Olympians: The Lightning Thief',
 'The Social Network',
 "Valentine's Day",
 "The Sorcerer's Apprentice",
 'Due Date',
 'Eat Pray Love',
 'Yogi Bear',
 'Red',
 'Paranormal Activity 2',
 'The A-Team',
 'Jackass 3-D',
 'The Other Guys',
 'Unstoppable',
 'Step Up 3-D',
 'The Book of E

In [33]:
len(list(gross_df['title'].unique()))

3386

In [35]:
len(list(gross_df['title']))

3387

In [38]:
gross_df['title'].value_counts()

Bluebeard                  2
Brick Mansions             1
Hubble 3D                  1
Star Trek Into Darkness    1
A Simple Favor             1
                          ..
Picture Me                 1
A Simple Life              1
Kong: Skull Island         1
Passion                    1
Jack the Giant Slayer      1
Name: title, Length: 3386, dtype: int64

In [40]:
recent_gross_df = gross_df[gross_df['year'] >= 2013]
display(recent_gross_df.head())

,title,studio,domestic_gross,foreign_gross,year
1127,Frozen,BV,400700000.0,875700000,2013
1128,Iron Man 3,BV,409000000.0,805800000,2013
1129,Despicable Me 2,Uni.,368100000.0,602700000,2013
1130,The Hobbit: The Desolation of Smaug,WB (NL),258399999.0,700000000,2013
1131,The Hunger Games: Catching Fire,LGF,424700000.0,440300000,2013


In [23]:
info_df

,id,synopsis,rating,genre,director,writer,theater_date,dvd_date,currency,box_office,runtime,studio
0,1,"This gritty, fast-paced, and innovative police...",R,Action and Adventure|Classics|Drama,William Friedkin,Ernest Tidyman,"Oct 9, 1971","Sep 25, 2001",NaN,NaN,104 minutes,NaN
1,3,"New York City, not-too-distant-future: Eric Pa...",R,Drama|Science Fiction and Fantasy,David Cronenberg,David Cronenberg|Don DeLillo,"Aug 17, 2012","Jan 1, 2013",$,"600,000",108 minutes,Entertainment One
2,5,Illeana Douglas delivers a superb performance ...,R,Drama|Musical and Performing Arts,Allison Anders,Allison Anders,"Sep 13, 1996","Apr 18, 2000",NaN,NaN,116 minutes,NaN
3,6,Michael Douglas runs afoul of a treacherous su...,R,Drama|Mystery and Suspense,Barry Levinson,Paul Attanasio|Michael Crichton,"Dec 9, 1994","Aug 27, 1997",NaN,NaN,128 minutes,NaN
4,7,NaN,NR,Drama|Romance,Rodney Bennett,Giles Cooper,NaN,NaN,NaN,NaN,200 minutes,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
1555,1996,Forget terrorists or hijackers -- there's a ha...,R,Action and Adventure|Horror|Mystery and Suspense,NaN,NaN,"Aug 18, 2006","Jan 2, 2007",$,"33,886,034",106 minutes,New Line Cinema
1556,1997,The popular Saturday Night Live sketch was exp...,PG,Comedy|Science Fiction and Fantasy,Steve Barron,Terry Turner|Tom Davis|Dan Aykroyd|Bonnie Turner,"Jul 23, 1993","Apr 17, 2001",NaN,NaN,88 minutes,Paramount Vantage
1557,1998,"Based on a novel by Richard Powell, when the l...",G,Classics|Comedy|Drama|Musical and Performing Arts,Gordon Douglas,NaN,"Jan 1, 1962","May 11, 2004",NaN,NaN,111 minutes,NaN
1558,1999,The Sandlot is a coming-of-age story about a g...,PG,Comedy|Drama|Kids and Family|Sports and Fitness,David Mickey Evans,David Mickey Evans|Robert Gunter,"Apr 1, 1993","Jan 29, 2002",NaN,NaN,101 minutes,NaN


In [24]:
reviews_df

,id,review,rating,fresh,critic,top_critic,publisher,date
0,3,A distinctly gallows take on contemporary fina...,3/5,fresh,PJ Nabarro,0,Patrick Nabarro,"November 10, 2018"
1,3,It's an allegory in search of a meaning that n...,NaN,rotten,Annalee Newitz,0,io9.com,"May 23, 2018"
2,3,... life lived in a bubble in financial dealin...,NaN,fresh,Sean Axmaker,0,Stream on Demand,"January 4, 2018"
3,3,Continuing along a line introduced in last yea...,NaN,fresh,Daniel Kasman,0,MUBI,"November 16, 2017"
4,3,... a perverse twist on neorealism...,NaN,fresh,NaN,0,Cinema Scope,"October 12, 2017"
...,...,...,...,...,...,...,...,...
54427,2000,The real charm of this trifle is the deadpan c...,NaN,fresh,Laura Sinagra,1,Village Voice,"September 24, 2002"
54428,2000,NaN,1/5,rotten,Michael Szymanski,0,Zap2it.com,"September 21, 2005"
54429,2000,NaN,2/5,rotten,Emanuel Levy,0,EmanuelLevy.Com,"July 17, 2005"
54430,2000,NaN,2.5/5,rotten,Christopher Null,0,Filmcritic.com,"September 7, 2003"


In [25]:
budgets_df

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross
0,1,"Dec 18, 2009",Avatar,"$425,000,000","$760,507,625","$2,776,345,279"
1,2,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,"$410,600,000","$241,063,875","$1,045,663,875"
2,3,"Jun 7, 2019",Dark Phoenix,"$350,000,000","$42,762,350","$149,762,350"
3,4,"May 1, 2015",Avengers: Age of Ultron,"$330,600,000","$459,005,868","$1,403,013,963"
4,5,"Dec 15, 2017",Star Wars Ep. VIII: The Last Jedi,"$317,000,000","$620,181,382","$1,316,721,747"
...,...,...,...,...,...,...
5777,78,"Dec 31, 2018",Red 11,"$7,000",$0,$0
5778,79,"Apr 2, 1999",Following,"$6,000","$48,482","$240,495"
5779,80,"Jul 13, 2005",Return to the Land of Wonders,"$5,000","$1,338","$1,338"
5780,81,"Sep 29, 2015",A Plague So Pleasant,"$1,400",$0,$0


In [47]:
len(movie_basics['primary_title'].unique())

136071

In [46]:
movie_basics['primary_title'].value_counts()

Home                    24
Broken                  20
The Return              20
Homecoming              16
Alone                   16
                        ..
Fields of November       1
Image of a Lost City     1
Pioneers' Palace         1
Horror Bathroom          1
Antikaci                 1
Name: primary_title, Length: 136071, dtype: int64

In [66]:
recent_gross_df.head()

,title,studio,domestic_gross,foreign_gross,year
1127,Frozen,BV,400700000.0,875700000,2013
1128,Iron Man 3,BV,409000000.0,805800000,2013
1129,Despicable Me 2,Uni.,368100000.0,602700000,2013
1130,The Hobbit: The Desolation of Smaug,WB (NL),258399999.0,700000000,2013
1131,The Hunger Games: Catching Fire,LGF,424700000.0,440300000,2013


In [77]:
recent_gross_df.sort_values(by='year',ascending=False)

,title,studio,domestic_gross,foreign_gross,year
3386,An Actor Prepares,Grav.,1700.0,NaN,2018
3183,On the Basis of Sex,Focus,24600000.0,13600000,2018
3176,Tyler Perry's Acrimony,LGF,43500000.0,2900000,2018
3177,Mary Queen of Scots,Focus,16500000.0,29900000,2018
3178,The Possession of Hannah Grace,SGem,14800000.0,28200000,2018
...,...,...,...,...,...
220,After.Life,Anch.,109000.0,1900000,2010
221,Cairo Time,IFC,1600000.0,391000,2010
222,Flipped,WB,1800000.0,NaN,2010
223,Guzaarish,UTV,1000000.0,695000,2010


In [76]:
movie_basics.sort_values(by='start_year', ascending=False).head(20)

,movie_id,primary_title,original_title,start_year,runtime_minutes,genres
89506,tt5174640,100 Years,100 Years,2115,NaN,Drama
96592,tt5637536,Avatar 5,Avatar 5,2027,NaN,"Action,Adventure,Fantasy"
2949,tt10300398,Untitled Star Wars Film,Untitled Star Wars Film,2026,NaN,Fantasy
52213,tt3095356,Avatar 4,Avatar 4,2025,NaN,"Action,Adventure,Fantasy"
105187,tt6149054,Fantastic Beasts and Where to Find Them 5,Fantastic Beasts and Where to Find Them 5,2024,NaN,"Adventure,Family,Fantasy"
2948,tt10300396,Untitled Star Wars Film,Untitled Star Wars Film,2024,NaN,None
2483,tt10255736,Untitled Marvel Project,Untitled Marvel Project,2023,NaN,Action
16337,tt1757678,Avatar 3,Avatar 3,2023,NaN,"Action,Adventure,Drama"
2906,tt10298848,Untitled Disney Live-Action Project,Untitled Disney Live-Action Project,2023,NaN,None
106865,tt6258542,Wraith of the Umbra and Eidolon II,Wraith of the Umbra and Eidolon II,2023,NaN,"Adventure,Drama,Fantasy"


In [ ]:
recent_imdb_movies = 

# Organized data wrangling
what dataframes are used and how we cleaned data

## SUMMARY
Our project will consider these datasets: 'tn.movie_budgets.csv.gz' and 'tmdb.movies.csv.gz' and 'im.db'


Our standard for "success" of a film is the money it makes (Revenue).
Variables that may influence the "success" of a film:

- Director
- Domestic Gross Revenue
- Worldwide Gross Revenue
- Average Vote Rating
- Vote Count
- Popularity
- Release Date
- Film Title
- Genre


Project will:

In [86]:
movies_df = pd.read_csv('data/zipped_files/tmdb.movies.csv.gz')
display(movies_df.head())
display(movies_df.info())

,Unnamed: 0,genre_ids,id,original_language,original_title,popularity,release_date,title,vote_average,vote_count
0,0,"[12, 14, 10751]",12444,en,Harry Potter and the Deathly Hallows: Part 1,33.533,2010-11-19,Harry Potter and the Deathly Hallows: Part 1,7.7,10788
1,1,"[14, 12, 16, 10751]",10191,en,How to Train Your Dragon,28.734,2010-03-26,How to Train Your Dragon,7.7,7610
2,2,"[12, 28, 878]",10138,en,Iron Man 2,28.515,2010-05-07,Iron Man 2,6.8,12368
3,3,"[16, 35, 10751]",862,en,Toy Story,28.005,1995-11-22,Toy Story,7.9,10174
4,4,"[28, 878, 12]",27205,en,Inception,27.920,2010-07-16,Inception,8.3,22186


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26517 entries, 0 to 26516
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Unnamed: 0         26517 non-null  int64  
 1   genre_ids          26517 non-null  object 
 2   id                 26517 non-null  int64  
 3   original_language  26517 non-null  object 
 4   original_title     26517 non-null  object 
 5   popularity         26517 non-null  float64
 6   release_date       26517 non-null  object 
 7   title              26517 non-null  object 
 8   vote_average       26517 non-null  float64
 9   vote_count         26517 non-null  int64  
dtypes: float64(2), int64(3), object(5)
memory usage: 2.0+ MB


None

,movie_id,primary_title,original_title,start_year,runtime_minutes,genres
0,tt0063540,Sunghursh,Sunghursh,2013,175.0,"Action,Crime,Drama"
1,tt0066787,One Day Before the Rainy Season,Ashad Ka Ek Din,2019,114.0,"Biography,Drama"
2,tt0069049,The Other Side of the Wind,The Other Side of the Wind,2018,122.0,Drama
3,tt0069204,Sabse Bada Sukh,Sabse Bada Sukh,2018,NaN,"Comedy,Drama"
4,tt0100275,The Wandering Soap Opera,La Telenovela Errante,2017,80.0,"Comedy,Drama,Fantasy"


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 146144 entries, 0 to 146143
Data columns (total 6 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   movie_id         146144 non-null  object 
 1   primary_title    146144 non-null  object 
 2   original_title   146123 non-null  object 
 3   start_year       146144 non-null  int64  
 4   runtime_minutes  114405 non-null  float64
 5   genres           140736 non-null  object 
dtypes: float64(1), int64(1), object(4)
memory usage: 6.7+ MB


None

# Cleaning dataframes and making new ones

## Cleaning movie_aka

In [ ]:
movie_akas = pd.read_sql("""

SELECT *
FROM movie_akas

""", conn)

display(movie_akas.head())
display(movie_akas.info())

In [56]:
imdb_all_movie_names = pd.read_sql("""

SELECT DISTINCT l.movie_id, l.title, r.primary_title, r.original_title
FROM movie_akas AS l
INNER JOIN movie_basics AS r
    ON l.movie_id = r.movie_id

""",conn)

imdb_all_movie_names

,movie_id,title,primary_title,original_title
0,tt0369610,Джурасик свят,Jurassic World,Jurassic World
1,tt0369610,Jurashikku warudo,Jurassic World,Jurassic World
2,tt0369610,Jurassic World: O Mundo dos Dinossauros,Jurassic World,Jurassic World
3,tt0369610,O Mundo dos Dinossauros,Jurassic World,Jurassic World
4,tt0369610,Jurassic World,Jurassic World,Jurassic World
...,...,...,...,...
266864,tt9755806,Большая Акула,Big Shark,Big Shark
266865,tt9827784,Sayonara kuchibiru,Farewell Song,Sayonara kuchibiru
266866,tt9827784,Farewell Song,Farewell Song,Sayonara kuchibiru
266867,tt9880178,La atención,The Attention,La atención


In [87]:
#taking out only movie_id and title columns from movie_akas
movie_akas = pd.read_sql("""

SELECT movie_id, title
FROM movie_akas

""", conn)
movie_akas

,movie_id,title
0,tt0369610,Джурасик свят
1,tt0369610,Jurashikku warudo
2,tt0369610,Jurassic World: O Mundo dos Dinossauros
3,tt0369610,O Mundo dos Dinossauros
4,tt0369610,Jurassic World
...,...,...
331698,tt9827784,Sayonara kuchibiru
331699,tt9827784,Farewell Song
331700,tt9880178,La atención
331701,tt9880178,La atención


In [88]:
# Grouping all movie_akas using the movie_id
movie_akas.groupby(by='movie_id').agg(lambda x: list(x))

,title
movie_id,
tt0063540,"[Sangharsh, Sunghursh, Sunghursh, Sunghursh, S..."
tt0066787,"[One Day Before the Rainy Season, Ashad Ka Ek ..."
tt0069049,"[La otra cara del viento, Druga strona wiatru,..."
tt0069204,"[Subse Bada Sukh, The Greatest Happiness, Sabs..."
tt0100275,"[La Telenovela Errante, The Wandering Soap Ope..."
...,...
tt9911774,"[Padmavyuhathile Abhimanyu, Pathmavyuhathile A..."
tt9913248,"[Nepal - Homebird, Nepal - Homebird, Népal - À..."
tt9914254,"[Kirsebæreventyret, A Cherry Tale, Kirsebæreve..."


In [ ]:
#trying to find a way to see if titles in budgets_df matches any in the lists from groupby in movies_aka

movie_akas['movie_id'].loc[title in titles for titles in movie_akas['titles'] for title in budgets_df['movie']]

for title in budgets_df['movie']:
    for titles in movie_akas['titles']:
        print(title in titles)

## combining persons and directors so that we know person's name

In [95]:
display(persons.head())
display(persons.info())

,person_id,primary_name,birth_year,death_year,primary_profession
0,nm0061671,Mary Ellen Bauder,NaN,NaN,"miscellaneous,production_manager,producer"
1,nm0061865,Joseph Bauer,NaN,NaN,"composer,music_department,sound_department"
2,nm0062070,Bruce Baum,NaN,NaN,"miscellaneous,actor,writer"
3,nm0062195,Axel Baumann,NaN,NaN,"camera_department,cinematographer,art_department"
4,nm0062798,Pete Baxter,NaN,NaN,"production_designer,art_department,set_decorator"


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 606648 entries, 0 to 606647
Data columns (total 5 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   person_id           606648 non-null  object 
 1   primary_name        606648 non-null  object 
 2   birth_year          82736 non-null   float64
 3   death_year          6783 non-null    float64
 4   primary_profession  555308 non-null  object 
dtypes: float64(2), object(3)
memory usage: 23.1+ MB


None

In [96]:
display(directors.head())
display(directors.info())

,movie_id,person_id
0,tt0285252,nm0899854
1,tt0462036,nm1940585
2,tt0835418,nm0151540
3,tt0835418,nm0151540
4,tt0878654,nm0089502


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 291174 entries, 0 to 291173
Data columns (total 2 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   movie_id   291174 non-null  object
 1   person_id  291174 non-null  object
dtypes: object(2)
memory usage: 4.4+ MB


None

In [ ]:
directors_names = pd.read_sql("""

SELECT DISTINCT l.person_id, r.primary_title, r.original_title
FROM directors AS l
INNER JOIN movie_basics AS r
    ON l.movie_id = r.movie_id

""",conn)

imdb_all_movie_names

In [105]:
imdb_directors = pd.read_sql("""

SELECT d.movie_id, p.primary_name 
FROM directors as d
LEFT JOIN persons AS p
    ON d.person_id = p.person_id
GROUP BY d.movie_id

""", conn)

imdb_directors.head()

,movie_id,primary_name
0,tt0063540,Harnam Singh Rawail
1,tt0066787,Mani Kaul
2,tt0069049,Orson Welles
3,tt0069204,Hrishikesh Mukherjee
4,tt0100275,Valeria Sarmiento


## Cleaning movie_basics

In [89]:
movie_basics = pd.read_sql("""

SELECT *
FROM movie_basics

""", conn)

display(movie_basics.head())
display(movie_basics.info())

,movie_id,primary_title,original_title,start_year,runtime_minutes,genres
0,tt0063540,Sunghursh,Sunghursh,2013,175.0,"Action,Crime,Drama"
1,tt0066787,One Day Before the Rainy Season,Ashad Ka Ek Din,2019,114.0,"Biography,Drama"
2,tt0069049,The Other Side of the Wind,The Other Side of the Wind,2018,122.0,Drama
3,tt0069204,Sabse Bada Sukh,Sabse Bada Sukh,2018,NaN,"Comedy,Drama"
4,tt0100275,The Wandering Soap Opera,La Telenovela Errante,2017,80.0,"Comedy,Drama,Fantasy"


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 146144 entries, 0 to 146143
Data columns (total 6 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   movie_id         146144 non-null  object 
 1   primary_title    146144 non-null  object 
 2   original_title   146123 non-null  object 
 3   start_year       146144 non-null  int64  
 4   runtime_minutes  114405 non-null  float64
 5   genres           140736 non-null  object 
dtypes: float64(1), int64(1), object(4)
memory usage: 6.7+ MB


None

In [92]:
len(movie_basics['movie_id'].unique())

146144

## Cleaning budgets_df

In [79]:
budgets_df = pd.read_csv('data/zipped_files/tn.movie_budgets.csv.gz')
display(budgets_df.head())
display(budgets_df.info())

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross
0,1,"Dec 18, 2009",Avatar,"$425,000,000","$760,507,625","$2,776,345,279"
1,2,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,"$410,600,000","$241,063,875","$1,045,663,875"
2,3,"Jun 7, 2019",Dark Phoenix,"$350,000,000","$42,762,350","$149,762,350"
3,4,"May 1, 2015",Avengers: Age of Ultron,"$330,600,000","$459,005,868","$1,403,013,963"
4,5,"Dec 15, 2017",Star Wars Ep. VIII: The Last Jedi,"$317,000,000","$620,181,382","$1,316,721,747"


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5782 entries, 0 to 5781
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   id                 5782 non-null   int64 
 1   release_date       5782 non-null   object
 2   movie              5782 non-null   object
 3   production_budget  5782 non-null   object
 4   domestic_gross     5782 non-null   object
 5   worldwide_gross    5782 non-null   object
dtypes: int64(1), object(5)
memory usage: 271.2+ KB


None

In [84]:
#getting rid of dollar signs and commas in budgets_df and convertting budget_df['domestic_gross'] from str to int
budgets_df['production_budget'] = (budgets_df['production_budget'].str.replace('$', '')
                                   .str.replace(',', '').astype(int))
budgets_df['production_budget']

0       425000000
1       410600000
2       350000000
3       330600000
4       317000000
          ...    
5777         7000
5778         6000
5779         5000
5780         1400
5781         1100
Name: production_budget, Length: 5782, dtype: int64

In [80]:
#getting rid of dollar signs and commas in budgets_df and convertting budget_df['domestic_gross'] from str to int
budgets_df['domestic_gross'] = budgets_df['domestic_gross'].str.replace('$', '').str.replace(',', '').astype(int)
budgets_df['domestic_gross']

0       760507625
1       241063875
2        42762350
3       459005868
4       620181382
          ...    
5777            0
5778        48482
5779         1338
5780            0
5781       181041
Name: domestic_gross, Length: 5782, dtype: int64

In [81]:
#getting rid of dollar signs and commas in budgets_df and convertting budget_df['domestic_gross'] from str to int
budgets_df['worldwide_gross'] = budgets_df['worldwide_gross'].str.replace('$', '').str.replace(',', '').astype(int)
budgets_df['worldwide_gross']

0       2776345279
1       1045663875
2        149762350
3       1403013963
4       1316721747
           ...    
5777             0
5778        240495
5779          1338
5780             0
5781        181041
Name: worldwide_gross, Length: 5782, dtype: int64

In [85]:
#making a foreign_gross column
budgets_df['foreign_gross'] = budgets_df['worldwide_gross'] - budgets_df['domestic_gross']
budgets_df.head()

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross,foreign_gross
0,1,"Dec 18, 2009",Avatar,425000000,760507625,2776345279,2015837654
1,2,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,410600000,241063875,1045663875,804600000
2,3,"Jun 7, 2019",Dark Phoenix,350000000,42762350,149762350,107000000
3,4,"May 1, 2015",Avengers: Age of Ultron,330600000,459005868,1403013963,944008095
4,5,"Dec 15, 2017",Star Wars Ep. VIII: The Last Jedi,317000000,620181382,1316721747,696540365


In [97]:
#import datetime
from datetime import datetime

In [104]:
# convert budgets_df['release_date'] to date time object 
budgets_df['clean_release_date'] =  pd.to_datetime(budgets_df['release_date'], infer_datetime_format=True)
budgets_df.head()

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross,foreign_gross,clean_release_date
0,1,"Dec 18, 2009",Avatar,425000000,760507625,2776345279,2015837654,2009-12-18
1,2,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,410600000,241063875,1045663875,804600000,2011-05-20
2,3,"Jun 7, 2019",Dark Phoenix,350000000,42762350,149762350,107000000,2019-06-07
3,4,"May 1, 2015",Avengers: Age of Ultron,330600000,459005868,1403013963,944008095,2015-05-01
4,5,"Dec 15, 2017",Star Wars Ep. VIII: The Last Jedi,317000000,620181382,1316721747,696540365,2017-12-15
